<a href="https://colab.research.google.com/github/MostafaShehata8/CV_MJ/blob/main/Vision_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **install dependencies**

In [31]:
!pip install tensorflow-datasets
!pip install opencv-python-headless
!pip install scikit-image

In [32]:
import cv2
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from skimage import exposure
import tensorflow_datasets as tfds

# **Load Dataset**

In [33]:

# Load the Caltech-101 dataset
dataset, info = tfds.load('caltech101', with_info=True, as_supervised=True)

# Get the training and test datasets
train_dataset, test_dataset = dataset['train'], dataset['test']


# **Hog Feature**

In [51]:
from skimage.feature import hog

def extract_hog_features(image):
    """
    Extracts HOG features from the input image.
    """
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute the HOG features
    fd, hog_image = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

    # Return both the HOG features (fd) and the HOG image (hog_image)
    return fd, hog_image



In [52]:
from skimage import exposure
import matplotlib.pyplot as plt

def visualize_hog(image):
    """
    Visualizes HOG features extracted from the input image.
    """
    # Extract HOG features
    fd, hog_image = extract_hog_features(image)

    # Rescale the intensity of the HOG image for better visibility
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    # Plot the HOG image
    plt.imshow(hog_image_rescaled, cmap=plt.cm.gray)
    plt.title("HOG Features")
    plt.axis("off")
    plt.show()





In [ ]:
def get_image_at_index(dataset, index):
    """
    Get an image from the dataset at a specific index.
    """
    image, label = list(dataset)[index]
    return np.array(image)

# Pick the 20th image (change the index as needed)
example_image = get_image_at_index(train_dataset, 50)
plt.title("Original Image")
plt.imshow(example_image)
plt.show()

# Visualize the HOG features for the selected image
visualize_hog(example_image)




# **Color Histogram**


In [25]:
def extract_color_histogram(image):
    # Convert image to RGB if it's not
    if len(image.shape) == 2:  # If grayscale, convert to BGR
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Calculate histograms for each color channel (B, G, R)
    hist_b = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([image], [2], None, [256], [0, 256])

    # Normalize the histograms
    hist_b /= hist_b.sum()
    hist_g /= hist_g.sum()
    hist_r /= hist_r.sum()

    # Concatenate the histograms of the three channels
    hist = np.concatenate([hist_b, hist_g, hist_r])

    return hist


# **local binary patterns histogram**

In [16]:
def extract_lbp_histogram(image, radius=1, n_points=8):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply LBP
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')

    # Calculate histogram of LBP
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))

    # Normalize the histogram
    lbp_hist = lbp_hist.astype('float')
    lbp_hist /= (lbp_hist.sum() + 1e-6)

    return lbp_hist


# **Combine features**

In [17]:
def extract_features(image):
    # Resize image to a standard size
    image = cv2.resize(image, (224, 224))

    # Extract HOG features
    hog_features = extract_hog_features(image)

    # Extract color histogram
    color_hist = extract_color_histogram(image)

    # Extract LBP histogram
    lbp_hist = extract_lbp_histogram(image)

    # Combine all features into a single vector
    features = np.concatenate([hog_features.flatten(), color_hist.flatten(), lbp_hist.flatten()])

    return features


In [ ]:
def process_dataset(dataset):
    features = []
    labels = []

    for image, label in dataset:
        image = np.array(image)
        feature_vector = extract_features(image)

        features.append(feature_vector)
        labels.append(label.numpy())

    return np.array(features), np.array(labels)

# Process the training set
train_features, train_labels = process_dataset(train_dataset)

print("Features shape:", train_features.shape)
print("Labels shape:", train_labels.shape)
